In [168]:
import pandas as pd
import numpy as np

from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score

import lightgbm as lgb

from lightgbm import LGBMRegressor
from sklearn.metrics import mean_squared_error, r2_score

In [169]:
df = pd.read_csv("../data/훈련데이터셋/3_훈련데이터셋_LAG_휴일포함.csv")
#df.info()

In [170]:
df.tail(5)

,날짜,전체코드,최저기온(°C),0.5m 지중온도(°C),평균 증기압(hPa),가조시간(hr),평균 상대습도(%),파워,lag_1d,lag_7d,휴일여부
233203,2024-01-24,1174011000,-11.2,2.3,1.7,10.1,52.0,115124.547,114960.859,108004.536,False
233204,2024-01-25,1174011000,-9.7,2.2,2.0,10.1,47.8,99907.500,115124.547,98758.864,False
233205,2024-01-26,1174011000,-6.9,2.1,3.4,10.1,62.5,94818.512,99907.500,NaN,False
233206,2024-01-27,1174011000,-4.6,2.0,4.0,10.2,70.0,86792.828,94818.512,76691.460,True
233207,2024-01-28,1174011000,-5.0,1.9,3.4,10.2,62.3,86389.477,86792.828,89640.934,True


In [171]:
df["휴일여부"] = df["휴일여부"].astype(int)

In [172]:
# 3일간 평균 최저기온
df["temp_3d_mean"] = (
    df.groupby("전체코드")["최저기온(°C)"]
      .shift(1).rolling(3).mean()
)

In [173]:
# 기온 변동성
df["temp_7d_std"] = (
    df.groupby("전체코드")["최저기온(°C)"]
      .shift(1)
      .rolling(7)
      .std()
)

In [174]:
# 단기 변동성
df["temp_3d_std"] = (
    df.groupby("전체코드")["최저기온(°C)"]
      .shift(1)
      .rolling(3)
      .std()
)

In [175]:
feature_cols = [
    # 지역
    "전체코드",

    # 날씨
    "temp_7d_std",
    "temp_3d_mean",
    "temp_3d_std",
    "최저기온(°C)", "0.5m 지중온도(°C)", "평균 증기압(hPa)","가조시간(hr)","평균 상대습도(%)",

    # 기간

    "lag_1d", "lag_7d"

]
target_col = "파워"

In [176]:
feature_cols = feature_cols + ["휴일여부"]

In [177]:
weather_cols = [
    "최저기온(°C)",
    "0.5m 지중온도(°C)",
    "평균 증기압(hPa)",
    "가조시간(hr)",
    "평균 상대습도(%)"
]

In [178]:
# new_df = df.dropna(subset=["파워"]).reset_index(drop=True)

In [179]:
new_df = df[["날짜"] + feature_cols + [target_col]].dropna().reset_index(drop=True)

In [180]:
# print(len(df), len(new_df))
# new_df.isna().sum().sort_values(ascending=False)

In [181]:
# =========================
# 1) 시간순 Train/Valid/Test 분할
# =========================
new_df = new_df.sort_values("날짜").reset_index(drop=True)

n = len(new_df)
test_size = int(n * 0.20)
valid_size = int(n * 0.10)

train_end = n - (valid_size + test_size)
valid_end = n - test_size

train_df = new_df.iloc[:train_end]
valid_df = new_df.iloc[train_end:valid_end]
test_df  = new_df.iloc[valid_end:]

In [182]:
# =========================
# 2) X / y 분리
# =========================
drop_cols = [target_col, "날짜"]  # 날짜는 직접 입력 피처로 쓰지 않음(우린 파생변수로 이미 반영)
X_train = train_df.drop(columns=drop_cols)
y_train = train_df[target_col]

X_valid = valid_df.drop(columns=drop_cols)
y_valid = valid_df[target_col]

X_test = test_df.drop(columns=drop_cols)
y_test = test_df[target_col]

In [183]:

# =========================
# 3) 범주형 처리 (코드형 컬럼은 category로)
# =========================
cat_cols = []
for c in ["전체코드", "시군구코드", "법정동코드"]:
    if c in X_train.columns:
        cat_cols.append(c)

for c in cat_cols:
    X_train[c] = X_train[c].astype("category")
    X_valid[c] = X_valid[c].astype("category")
    X_test[c]  = X_test[c].astype("category")

In [184]:
# =========================
# 4) LightGBM Baseline 모델
# =========================
model = LGBMRegressor(
    objective="regression_l2",   # 🔥 반드시 변경

    n_estimators=20000,
    learning_rate=0.03,

    num_leaves=63,
    max_depth=10,
    min_child_samples=80,        # ↓ 완화
    min_split_gain=0.05,         # ↓ 완화

    subsample=0.8,
    subsample_freq=1,
    colsample_bytree=0.8,

    reg_alpha=0.1,               # ↓
    reg_lambda=5.0,              # ↓

    random_state=42,
    n_jobs=-1
)

model.fit(
    X_train, y_train,
    eval_set=[(X_valid, y_valid)],
    eval_metric="rmse",
    categorical_feature=cat_cols if len(cat_cols) > 0 else "auto",
    callbacks=[
        # early stopping (너무 오래 돌지 않게)
        # verbose는 200번마다 한번씩 로그
        __import__("lightgbm").early_stopping(stopping_rounds=200, verbose=True),
        __import__("lightgbm").log_evaluation(period=200),
    ]
)

[LightGBM] [Warning] Categorical features with more bins than the configured maximum bin number found.
[LightGBM] [Warning] For categorical features, max_bin and max_bin_by_feature may be ignored with a large number of categories.
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.003948 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2463
[LightGBM] [Info] Number of data points in the train set: 91702, number of used features: 12
[LightGBM] [Warning] Found whitespace in feature_names, replace with underlines
[LightGBM] [Info] Start training from score 132194.274783
Training until validation scores don't improve for 200 rounds
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positive gain, best gain: -inf
[LightGBM] [Warning] No further splits with positi

LGBMRegressor(colsample_bytree=0.8, learning_rate=0.03, max_depth=10,
              min_child_samples=80, min_split_gain=0.05, n_estimators=20000,
              n_jobs=-1, num_leaves=63, objective='regression_l2',
              random_state=42, reg_alpha=0.1, reg_lambda=5.0, subsample=0.8,
              subsample_freq=1)

In [185]:
# =========================
# 5) 평가 (Valid / Test)
# =========================
def eval_print(name, y_true, y_pred):
    rmse = mean_squared_error(y_true, y_pred, squared=False)
    r2 = r2_score(y_true, y_pred)
    print(f"[{name}] RMSE: {rmse:,.3f}")
    print(f"[{name}] R2  : {r2:.6f}")
    return rmse, r2

pred_valid = model.predict(X_valid)
pred_test  = model.predict(X_test)

eval_print("VALID", y_valid, pred_valid)
eval_print("TEST",  y_test,  pred_test)

print("Best iteration:", model.best_iteration_)

[VALID] RMSE: 217,490.976
[VALID] R2  : 0.728935
[TEST] RMSE: 68,013.218
[TEST] R2  : 0.896395
Best iteration: 1484


In [186]:
from sklearn.metrics import mean_squared_error, mean_absolute_error

# =========================
# 예측
# =========================
y_train_pred = model.predict(X_train)
y_valid_pred = model.predict(X_valid)
y_test_pred  = model.predict(X_test)

# =========================
# MSE / MAE 계산
# =========================
metrics = {
    "TRAIN": {
        "MSE": mean_squared_error(y_train, y_train_pred),
        "MAE": mean_absolute_error(y_train, y_train_pred)
    },
    "VALID": {
        "MSE": mean_squared_error(y_valid, y_valid_pred),
        "MAE": mean_absolute_error(y_valid, y_valid_pred)
    },
    "TEST": {
        "MSE": mean_squared_error(y_test, y_test_pred),
        "MAE": mean_absolute_error(y_test, y_test_pred)
    }
}

# =========================
# 출력
# =========================
for k, v in metrics.items():
    print(f"[{k}]")
    print(f"  MSE : {v['MSE']:,.3f}")
    print(f"  MAE : {v['MAE']:,.3f}")

[TRAIN]
  MSE : 6,166,330,368.334
  MAE : 7,956.608
[VALID]
  MSE : 47,302,324,535.054
  MAE : 16,442.402
[TEST]
  MSE : 4,625,797,869.845
  MAE : 11,949.486


In [187]:
# 전체 중요도



In [188]:
# 중요도 전체

import pandas as pd

imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

imp.head(20)

,feature,importance
0,전체코드,12346
10,lag_7d,9860
9,lag_1d,9747
1,temp_7d_std,5614
3,temp_3d_std,4963
8,평균 상대습도(%),4686
5,0.5m 지중온도(°C),4212
2,temp_3d_mean,3873
7,가조시간(hr),3837
4,최저기온(°C),3737


In [189]:
# 전체 중요도
import pandas as pd

imp = pd.DataFrame({
    "feature": X_train.columns,
    "importance": model.feature_importances_
}).sort_values("importance", ascending=False)

imp["importance_pct"] = imp["importance"] / imp["importance"].sum() * 100

imp.head(20)

,feature,importance,importance_pct
0,전체코드,12346,18.148676
10,lag_7d,9860,14.494245
9,lag_1d,9747,14.328134
1,temp_7d_std,5614,8.252606
3,temp_3d_std,4963,7.295633
8,평균 상대습도(%),4686,6.888441
5,0.5m 지중온도(°C),4212,6.191659
2,temp_3d_mean,3873,5.693328
7,가조시간(hr),3837,5.640407
4,최저기온(°C),3737,5.493407


In [190]:
imp[imp["feature"] == "휴일여부"]

,feature,importance,importance_pct
11,휴일여부,1597,2.347597


In [191]:
imp[imp["feature"] == "전체코드"]

,feature,importance,importance_pct
0,전체코드,12346,18.148676


In [192]:
import pandas as pd
from sklearn.metrics import r2_score

# TEST 데이터 복사
test_df = X_test.copy()
test_df["y_true"] = y_test.values
test_df["y_pred"] = pred_test

# 법정동코드 포함 (X_test에 이미 있어야 함)
test_df["전체코드"] = df.loc[X_test.index, "전체코드"].values

In [193]:
dong_r2 = (
    test_df
    .groupby("전체코드")
    .apply(lambda g: r2_score(g["y_true"], g["y_pred"]))
    .reset_index(name="R2")
)

In [194]:
dong_cnt = (
    test_df
    .groupby("전체코드")
    .size()
    .reset_index(name="n_samples")
)

dong_eval = dong_r2.merge(dong_cnt, on="전체코드")

dong_eval.sort_values("R2").head(10)

,전체코드,R2,n_samples
1,1129010900,0.383939,580
2,1129011000,0.505759,580
0,1129010800,0.805502,226
4,1129011200,0.813117,580
22,1129013000,0.825556,580
29,1129013700,0.837107,580
20,1129012800,0.886526,580
28,1129013600,0.888188,580
30,1129013800,0.888572,580
21,1129012900,0.889416,580


In [195]:
dong_eval["R2"].describe()

count    46.000000
mean      0.924024
std       0.114671
min       0.383939
25%       0.928284
50%       0.964617
75%       0.976097
max       0.992827
Name: R2, dtype: float64

In [196]:
import joblib

# 저장
joblib.dump(model, "lgb_model.pkl")

# 불러오기
lgb_model = joblib.load("lgb_model.pkl")